In [7]:
!pip install pandas numpy matplotlib statsmodels scikit-learn

   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 7.8/7.8 MB 53.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   ---------------------------------------- 9.8/9.8 MB 61.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 11.6/11.6 MB 60.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 62.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


  You can safely remove it manually.


In [8]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Load the Chartered Bank Interest Rates data
chartered_df = pd.read_csv('chartered_bank_interest.csv', parse_dates=['date'])

# Load the Canada Bond Yield data
bond_yield_df = pd.read_csv('canada_bond_yield.csv', parse_dates=['date'])

# Display the first few rows of each DataFrame
print("Chartered Bank Interest Rates:")
print(chartered_df.head())

print("\nCanada Bond Yield:")
print(bond_yield_df.head())

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\meetr\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\meetr\AppData\Local\Temp\ipykernel_22904\2798944033.py", line 2, in <module>
    chartered_df = pd.read_csv('chartered_bank_interest.csv', parse_dates=['date'])
  File "c:\Users\meetr\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py", line 311, in wrapper
    and p.name not in allow_args
  File "c:\Users\meetr\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\parsers\readers.py", line 586, in read_csv
    """Generic reader of line files."""
  File "c:\Users\meetr\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\parsers\readers.py", line 482, in _read
  File "c:\Users\meetr\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\parsers\readers.py", line 811, in __init__
  

In [ ]:
# Check for missing values in chartered_df
print("\nMissing values in Chartered Bank Interest Rates:")
print(chartered_df.isnull().sum())

# Check for missing values in bond_yield_df
print("\nMissing values in Canada Bond Yield:")
print(bond_yield_df.isnull().sum())

# Fill missing values if any (e.g., forward fill)
chartered_df.fillna(method='ffill', inplace=True)
bond_yield_df.fillna(method='ffill', inplace=True)

In [ ]:
# Focus on the 5-year conventional mortgage rate (V80691335)
mortgage_df = chartered_df[['date', 'V80691335']].rename(columns={'V80691335': 'mortgage_rate'})

# Merge with bond yield data on date
merged_df = pd.merge(mortgage_df, bond_yield_df, on='date', how='left')

# Rename bond yield columns for clarity
merged_df.rename(columns={'value': 'bond_yield_value', 'market': 'bond_yield_market'}, inplace=True)

# Handle any remaining missing values after merge
merged_df.fillna(method='ffill', inplace=True)

# Set date as index
merged_df.set_index('date', inplace=True)

# Display the merged DataFrame
print("\nMerged DataFrame:")
print(merged_df.head())

In [ ]:
# Split the data into training and testing sets
train = merged_df.iloc[:-24]  # Use all data except the last 24 months for training
test = merged_df.iloc[-24:]   # Last 24 months for testing

# Define the SARIMA model
# You may need to adjust the order based on ACF and PACF plots
model = SARIMAX(train['mortgage_rate'],
                order=(1, 1, 1),
                seasonal_order=(1, 1, 1, 12),
                exog=train[['bond_yield_value', 'bond_yield_market']])

# Fit the model
model_fit = model.fit(disp=False)
print("\nModel Summary:")
print(model_fit.summary())

In [ ]:
# Plot diagnostics
model_fit.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# Forecast the next 24 months
forecast = model_fit.get_forecast(steps=24, exog=test[['bond_yield_value', 'bond_yield_market']])

# Get confidence intervals
conf_int = forecast.conf_int()

# Create a DataFrame for predicted values
forecast_df = forecast.predicted_mean
forecast_df = forecast_df.rename('forecast')

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(train.index, train['mortgage_rate'], label='Training')
plt.plot(test.index, test['mortgage_rate'], label='Actual', color='blue')
plt.plot(forecast_df.index, forecast_df, label='Forecast', color='red')
plt.fill_between(conf_int.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.xlabel('Date')
plt.ylabel('5-Year Fixed Mortgage Rate')
plt.title('Mortgage Rate Forecast')
plt.legend()
plt.show()

# Evaluate the model
predictions = forecast.predicted_mean
mae = mean_absolute_error(test['mortgage_rate'], predictions)
rmse = np.sqrt(mean_squared_error(test['mortgage_rate'], predictions))
print(f"\nMean Absolute Error: {mae:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")

In [ ]:
# Assuming you have bond yield forecasts for the next 24 months
# bond_yield_future = pd.read_csv('bond_yield_future.csv', parse_dates=['date'])

# Example: Using the last available bond yield data for future exogenous variables
bond_yield_future = pd.DataFrame({
    'date': pd.date_range(start=merged_df.index[-1] + pd.DateOffset(months=1), periods=24, freq='M'),
    'bond_yield_value': [merged_df['bond_yield_value'].iloc[-1]] * 24,
    'bond_yield_market': [merged_df['bond_yield_market'].iloc[-1]] * 24
})
bond_yield_future.set_index('date', inplace=True)

# Forecast mortgage rates
future_forecast = model_fit.get_forecast(steps=24, exog=bond_yield_future[['bond_yield_value', 'bond_yield_market']])
future_pred = future_forecast.predicted_mean
future_conf_int = future_forecast.conf_int()

# Plot future forecasts
plt.figure(figsize=(12, 6))
plt.plot(merged_df.index, merged_df['mortgage_rate'], label='Historical')
plt.plot(future_pred.index, future_pred, label='Future Forecast', color='green')
plt.fill_between(future_conf_int.index, future_conf_int.iloc[:, 0], future_conf_int.iloc[:, 1], color='lightgreen', alpha=0.3)
plt.xlabel('Date')
plt.ylabel('5-Year Fixed Mortgage Rate')
plt.title('Future Mortgage Rate Forecast')
plt.legend()
plt.show()

# Save the forecast to a CSV file
future_forecast_df = pd.DataFrame({
    'forecast': future_pred,
    'lower_ci': future_conf_int.iloc[:, 0],
    'upper_ci': future_conf_int.iloc[:, 1]
})
future_forecast_df.to_csv('future_mortgage_rate_forecast.csv')